## Análise do Comércio Exterior Brasileiro: Impactos das Novas Taxações e a Dependência em Relação aos EUA

O comércio exterior brasileiro é fortemente influenciado pelas relações com parceiros estratégicos, especialmente os Estados Unidos. Em um contexto de instabilidade global, a possibilidade de **novas taxações sobre produtos comercializados entre Brasil e EUA** levanta preocupações sobre os impactos para diversos setores econômicos e regiões do país.

Os EUA representam um dos principais destinos das exportações brasileiras, abrangendo desde commodities agrícolas e minerais até produtos manufaturados de alto valor agregado. Em contrapartida, o Brasil importa dos EUA tecnologias, bens de capital e insumos essenciais. Essa **interdependência comercial consolidada ao longo de décadas** sustenta cadeias produtivas em diferentes níveis — nacional, estadual e local.

A introdução de **novas tarifas, impostos ou barreiras não-tarifárias** pode provocar impactos significativos, tais como:
- Aumento dos custos de produção;
- Redução da competitividade de produtos brasileiros;
- Reorganização das cadeias de suprimento e produção;
- Busca por mercados alternativos para exportação e importação.

Compreender a **extensão dessa dependência** e os riscos associados é essencial para formular políticas públicas eficazes, desenvolver estratégias empresariais resilientes e promover a diversificação de mercados externos.

Esta análise explora:
- A importância estratégica dos EUA no comércio exterior brasileiro;
- Os possíveis efeitos de um cenário com novas taxações;
- Caminhos para **reduzir vulnerabilidades e aproveitar oportunidades** em meio às transformações do cenário internacional.

______________________________

## Apache Spark

Neste exercício, utilizaremos a plataforma **Google Colab** com **Apache Spark**, uma poderosa ferramenta de processamento de dados distribuído, por meio da sua API em Python chamada **PySpark**. Essa escolha se justifica pela necessidade de manipular dados em escala maior do que a memória de uma única máquina comportaria, com desempenho superior ao de bibliotecas tradicionais como Pandas.

### Vantagens do Apache Spark

- **Alta Velocidade**: Processamento em memória (in-memory), muito mais rápido que ferramentas baseadas em disco, como o Hadoop MapReduce.
- **Escalabilidade Horizontal**: Funciona em clusters, permitindo distribuir a carga entre várias máquinas.
- **API Unificada e Flexível**: Suporta diversas linguagens (Python, Scala, Java, R, SQL) e múltiplos tipos de processamento:
  - **Batch**
  - **Streaming (micro-batching)**
  - **Machine Learning (MLlib)**
  - **Grafos (GraphX)**
- **Tolerância a Falhas**: RDDs permitem recuperação automática em caso de falhas.
- **Sintaxe Familiar (PySpark)**: Interface semelhante ao Pandas, facilitando a adoção por quem já trabalha com Python.

### Desvantagens do Spark

- **Complexidade de Gerenciamento**: Requer conhecimento sobre clusters e configuração.
- **Custo Elevado**: Especialmente em ambientes de nuvem com grande volume de dados.
- **Streaming com Micro-batches**: Não é ideal para latência em milissegundos.
- **Curva de Aprendizado**: Debug e tuning podem ser desafiadores.

### Quando Usar Spark

- Para **grandes volumes de dados** que ultrapassam a memória local.
- Em **ETLs complexos**, algoritmos de machine learning e análise de grafos.
- Quando é necessário **processamento rápido e escalável**.
- Quando se deseja uma plataforma unificada para batch, streaming e ML.

### Considerações Técnicas sobre PySpark

- O Spark é escrito em **Scala** e roda na JVM.
- O **PySpark** atua como uma ponte: comandos Python são traduzidos e executados pelo engine do Spark em Scala/Java.
- Mesmo em ambientes como Google Colab, o Spark opera como um cluster local simplificado.

### Conclusão

Ao utilizar o Apache Spark com PySpark, combinamos a **sintaxe acessível do Python** com o **poder da computação distribuída**, ideal para manipulação de dados em larga escala. No entanto, seu uso deve ser cuidadosamente avaliado frente às necessidades e limitações do projeto.


### Instalar e importar bibliotecas e funções

In [ ]:
# Instalar
!pip install --quiet pyspark

In [ ]:
# Importar
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import requests
from datetime import datetime
from IPython.display import clear_output
import gc
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pyspark.sql.types as T
import subprocess

#### Conectar no google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Fonte dos Dados

Neste exercício, vamos trabalhar com os **dados brutos do comércio exterior brasileiro**, disponibilizados pelo **ComexStat**. Especificamente, utilizaremos os arquivos detalhados por Município da empresa exportadora/importadora e por **Posição do Sistema Harmonizado (SH4)**.

### Download e Estrutura dos Dados

Os arquivos estão disponíveis em formato **CSV**, com **separador ponto e vírgula (;)**, e incluem informações por:

- Ano e mês da operação
- Código SH4 do produto
- Código do país de destino (exportação) ou origem (importação)
- Código da UF e do município do domicílio fiscal da empresa
- Quilograma líquido
- Valor FOB em dólares (US$)

### Processamento com Apache Spark

Após o download, os dados serão **carregados e salvos como .parquet**. Isso nos permitirá processar eficientemente grandes volumes de dados de forma distribuída, com melhor desempenho e escalabilidade.

### Fonte oficial

Os arquivos podem ser acessados no portal oficial:

[ComexStat - Dados Brutos](https://www.gov.br/mdic/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta)



## Exportações: baixar dados e salvar como `.parquet`

In [ ]:
# #===============================================================================
# # Definir o intervalo de anos que serão baixados os dados
# #===============================================================================
# anos = range(2010, 2026)

# #===============================================================================
# # Baixar dados de exportação
# #===============================================================================
# # Inicializa uma lista vazia para armazenar os DataFrames
# dfs_exp = []

# for ano in anos:
#     # Constrói a URL dinamicamente para cada ano
#     url = f"https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/EXP_{ano}_MUN.csv"

#     try:
#         # Tenta ler o arquivo CSV da URL
#         print(f"Lendo dados de {ano}...")
#         df = pd.read_csv(url, sep=';')

#         # Adiciona o DataFrame recém-criado à lista 'dfs_exp'
#         dfs_exp.append(df)
#         print(f"-> Sucesso! DataFrame de {ano} adicionado à lista.")

#     except Exception as e:
#         # Informa caso o arquivo para um ano específico não seja encontrado ou ocorra outro erro
#         print(f"-> ERRO ao ler o arquivo para o ano {ano}. Motivo: {e}")

# # Unir dataframes
# df_comex_exp = pd.concat(dfs_exp, ignore_index=True)

# #===============================================================================
# # Baixar dados e Salvar como .parquet
# #===============================================================================
# # Para pandas DataFrame
# df_comex_exp.to_parquet("/content/drive/MyDrive/Projeto_comex_spark/data/dados_comex_mun_export_07-2025.parquet")

# print("Tabela criada e salva")

Tabela criada e salva


## Importações: baixar dados e salvar como `.parquet`

In [ ]:
# #===============================================================================
# # Definir o intervalo de anos que serão baixados os dados
# #===============================================================================
# anos = range(2010, 2026)

# #===============================================================================
# # Baixar dados de importações
# #===============================================================================
# # Inicializa uma lista vazia para armazenar os DataFrames
# dfs_imp = []

# for ano in anos:
#     # Constrói a URL dinamicamente para cada ano
#     url = f"https://balanca.economia.gov.br/balanca/bd/comexstat-bd/mun/IMP_{ano}_MUN.csv"

#     try:
#         # Tenta ler o arquivo CSV da URL
#         print(f"Lendo dados de {ano}...")
#         df = pd.read_csv(url, sep=';')

#         # Adiciona o DataFrame recém-criado à lista 'dfs_imp'
#         dfs_imp.append(df)
#         print(f"-> Sucesso! DataFrame de {ano} adicionado à lista.")

#     except Exception as e:
#         # Informa caso o arquivo para um ano específico não seja encontrado ou ocorra outro erro
#         print(f"-> ERRO ao ler o arquivo para o ano {ano}. Motivo: {e}")

# # Unir dataframes
# df_comex_imp = pd.concat(dfs_imp, ignore_index=True)

# #===============================================================================
# # Baixar dados e Salvar como .parquet
# #===============================================================================
# # Para pandas DataFrame
# df_comex_imp.to_parquet("/content/drive/MyDrive/Projeto_comex_spark/data/dados_comex_mun_import_07-2025.parquet")

# print("Tabela criada e salva")

## 📘 Dicionários e Classificações da Balança Comercial – COMEXSTAT

Este documento descreve os principais dicionários usados nas análises da balança comercial brasileira e como integrá-los usando a chave `CO_NCM`.

---

### Descrição dos Dicionários

### `NCM.csv` – Nomenclatura Comum do Mercosul
- Código de 8 dígitos que identifica mercadorias.
- Colunas principais:
  - `CO_NCM`: código NCM
  - `NO_NCM_POR`: descrição do produto
  - `CO_FAT_AGREG`: fator agregado (produto primário, consumo, etc.)

---

### `NCM_CGCE.csv` – Classificação por Grandes Categorias Econômicas
- Agrupa os produtos em categorias macroeconômicas como bens de consumo e bens de capital.
- Colunas:
  - `CO_CGCE_N1`, `CO_CGCE_N2`, `CO_CGCE_N3`
  - `NO_CGCE_N3`: descrição da categoria
  - `CO_NCM`: chave para unir

---

### `NCM_PPE.csv` – Pauta de Produtos Exportados
- Classificação temática para exportações.
- Colunas:
  - `CO_PPE`, `NO_PPE`
  - `CO_NCM`: chave para unir

---

### `NCM_PPI.csv` – Pauta de Produtos Importados
- Versão da PPE para importações.
- Colunas:
  - `CO_PPI`, `NO_PPI`
  - `CO_NCM`: chave para unir

---

### `NCM_UNIDADE.csv` – Unidades Estatísticas
- Define a unidade de medida usada para cada NCM.
- Colunas:
  - `CO_UNID`, `NO_UNID`
  - `CO_NCM`: chave para unir

---

### `CLASSIFICACAO-SEMANAL-ANTIGA_x_ISIC_x_CUCI-GRUPO.xlsx`
- Conversão entre classificações semanais antigas e sistemas internacionais ISIC e CUCI.
- Contém colunas como:
  - `SECAO_ISIC`, `GRUPO_CUCI`, `NO_CUCI`

---
### Esquema de JOINs entre os Dicionários COMEXSTAT

Abaixo, mostro como organizar os joins com base em suas chaves comuns. Após isso, apresento o código em Python para unir os arquivos de forma prática.

---

### 1. Dicionários baseados em NCM

Esses arquivos podem ser unidos usando `CO_NCM`:

- `NCM.csv` → Base principal, com a descrição do produto
- `NCM_SH.csv` → Liga NCM ao código SH
- `NCM_CUCI.csv` → Liga NCM à classificação CUCI
- `NCM_CGCE.csv` → Liga NCM à Classificação por Grandes Categorias Econômicas
- `NCM_PPE.csv` → Liga NCM à Pauta de Produtos Exportados
- `NCM_PPI.csv` → Liga NCM à Pauta de Produtos Importados
- `NCM_UNIDADE.csv` → Liga NCM à unidade estatística
- `NBM.csv` → Nomenclatura anterior (1989–1996)


## Baixar dicionários e planilhas auxiliares

In [ ]:
# # =============================================================================
# # Baixar tabelas e gerar dataframes
# # =============================================================================
# urls = {
#     "pais": "https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS.csv",
#     "bloco": "https://balanca.economia.gov.br/balanca/bd/tabelas/PAIS_BLOCO.csv",
#     "uf_mun": "https://balanca.economia.gov.br/balanca/bd/tabelas/UF_MUN.csv",
#     "uf": "https://balanca.economia.gov.br/balanca/bd/tabelas/UF.csv",
#     "via": "https://balanca.economia.gov.br/balanca/bd/tabelas/VIA.csv",
#     "urf": "https://balanca.economia.gov.br/balanca/bd/tabelas/URF.csv",
#     "ncm": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv",
#     "ncm_sh": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv",
#     "cuci": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CUCI.csv",
#     "isic_cuci": "https://balanca.economia.gov.br/balanca/bd/tabelas/ISIC_CUCI.csv",
#     "cgce": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CGCE.csv",
#     "fator_a": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_FAT_AGREG.csv",
#     "ppe": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPE.csv",
#     "ppi": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_PPI.csv",
#     "unid_estat": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_UNIDADE.csv",
#     "nbm_1996": "https://balanca.economia.gov.br/balanca/bd/tabelas/NBM.csv",
#     "nbm_x_ncm": "https://balanca.economia.gov.br/balanca/bd/tabelas/NBM_NCM.csv",
# }

# # Gerar dataframes automaticamente
# for nome, url in urls.items():
#     globals()[f'df_{nome}'] = pd.read_csv(url, sep=';', encoding='latin-1')
#     print(f"df_{nome}: {globals()[f'df_{nome}'].shape}")

# print("Geração dos dataframes concluída")

# # =============================================================================
# # UNIÃO POR GRUPOS TEMÁTICOS
# # =============================================================================

# # Geografia completa
# df_geo_master = df_uf_mun.merge(df_uf, on='SG_UF', how='left')

# #_______________________________________________________________

# # Países + blocos econômicos
# df_paises_master = df_pais.merge(df_bloco, on='CO_PAIS', how='left')

# # selecionar colunas uteis
# df_paises_master_select = df_paises_master[['CO_PAIS', 'CO_PAIS_ISON3', 'CO_PAIS_ISOA3', 'NO_PAIS', 'NO_BLOCO']]

# #_______________________________________________________________
# # NCM com todas as classificações (dataset principal)
# df_produtos_master = (df_ncm
#     .merge(df_ncm_sh, on='CO_SH6', how='left')
#     .merge(df_cuci, on='CO_CUCI_ITEM', how='left')
#     .merge(df_cgce, on='CO_CGCE_N3', how='left')
#     .merge(df_fator_a, on='CO_FAT_AGREG', how='left')
#     .merge(df_ppe, on='CO_PPE', how='left')
#     .merge(df_ppi, on='CO_PPI', how='left')
#     .merge(df_unid_estat, on='CO_UNID', how='left')
# )

# # selecionar colunas uteis
# df_produtos_master_select = df_produtos_master[['CO_SH4', 'NO_NCM_POR', 'NO_CUCI_GRUPO', 'NO_CUCI_DIVISAO', 'NO_PPE_MIN',
#                                                 'NO_CGCE_N2', 'NO_CGCE_N1', 'NO_FAT_AGREG', 'CO_CUCI_GRUPO']]

# #_______________________________________________________________
# # CUCI + ISIC
# df_cuci_master = df_cuci.merge(df_isic_cuci, on='CO_CUCI_GRUPO', how='left')

# # visualizar
# # Select only the necessary columns and drop the duplicate one before merging
# df_cuci_master_select = df_cuci_master[['NO_ISIC_SECAO', 'CO_CUCI_GRUPO']].drop_duplicates(subset=['CO_CUCI_GRUPO'])

# #_______________________________________________________________
# # df_produtos_master_select + df_cuci_master_select econômicos
# df_produtoscuci_master_select = df_produtos_master_select.merge(df_cuci_master_select, on='CO_CUCI_GRUPO', how='left')

# print("Combinação dos dicionários comcluída")

# # =============================================================================
# # SALVAR
# # =============================================================================
# # Lista completa com todos os DataFrames
# dataframes = {
#     # Novos dicionários das URLs
#     "pais": df_pais,
#     "bloco": df_bloco,
#     "uf_mun": df_uf_mun,
#     "uf": df_uf,
#     "via": df_via,
#     "urf": df_urf,
#     "ncm": df_ncm,
#     "ncm_sh": df_ncm_sh,
#     "cuci": df_cuci,
#     "isic_cuci": df_isic_cuci,
#     "cgce": df_cgce,
#     "fator_a": df_fator_a,
#     "ppe": df_ppe,
#     "ppi": df_ppi,
#     "unid_estat": df_unid_estat,
#     "nbm_1996": df_nbm_1996,
#     "nbm_x_ncm": df_nbm_x_ncm,

#     # Dicionários modificados
#     "df_geo_master"  : df_geo_master,
#     "df_paises_master" : df_paises_master,
#     "df_paises_master_select" : df_paises_master_select,
#     "df_produtos_master" : df_produtos_master,
#     "df_cuci_master" : df_cuci_master,
#     "df_cuci_master_select" : df_cuci_master_select,
#     "df_produtoscuci_master_select" : df_produtoscuci_master_select,
# }

# # Caminho base
# base_path = "/content/drive/MyDrive/Projeto_comex_spark/data/"

# # Criar diretório se não existir
# os.makedirs(base_path, exist_ok=True)

# # Salvar todos os DataFrames
# print("Salvando DataFrames...")
# for nome, df in dataframes.items():
#     try:
#         caminho_completo = f"{base_path}{nome}.parquet"
#         df.to_parquet(caminho_completo, index=False)
#         print(f"Salvo: {nome}.parquet")
#     except Exception as e:
#         print(f"Erro ao salvar {nome}: {e}")

# print(f"\n Processo concluído! Total de {len(dataframes)} DataFrames processados.")


## Defirnir import dos dados

In [ ]:
# Criar/reativar sessão Spark
spark = SparkSession.builder \
    .appName("COMEX Analysis") \
    .getOrCreate()

# Caminho base dos arquivos parquet
base_path = "/content/drive/MyDrive/Projeto_comex_spark/data/"

# Arquivos prontos para consultas
df_export = spark.read.parquet(f"{base_path}dados_comex_mun_export_07-2025.parquet")
df_import = spark.read.parquet(f"{base_path}dados_comex_mun_import_07-2025.parquet")

# Verificar estrutura (lazy evaluation - não carrega tudo)
print(f"\nEstrutura dos dados:")
print(f"Exportação: {df_export.count()} linhas")
print(f"Importação: {df_import.count()} linhas")

# Ver schema sem carregar dados
print("\nSchemas:")
df_export.printSchema()
df_import.printSchema()



Estrutura dos dados:
Exportação: 13779191 linhas
Importação: 22702125 linhas

Schemas:
root
 |-- CO_ANO: long (nullable = true)
 |-- CO_MES: long (nullable = true)
 |-- SH4: long (nullable = true)
 |-- CO_PAIS: long (nullable = true)
 |-- SG_UF_MUN: string (nullable = true)
 |-- CO_MUN: long (nullable = true)
 |-- KG_LIQUIDO: long (nullable = true)
 |-- VL_FOB: long (nullable = true)

root
 |-- CO_ANO: long (nullable = true)
 |-- CO_MES: long (nullable = true)
 |-- SH4: long (nullable = true)
 |-- CO_PAIS: long (nullable = true)
 |-- SG_UF_MUN: string (nullable = true)
 |-- CO_MUN: long (nullable = true)
 |-- KG_LIQUIDO: long (nullable = true)
 |-- VL_FOB: long (nullable = true)



## Análise Exploratória Simples

### **Período de dados disponíveis**

In [ ]:
inicio = time.time()

# EXPORTAÇÃO - Select apenas das colunas necessárias + criação da data temporária
print("=== EXPORTAÇÃO ===")
df_export.select("CO_ANO", "CO_MES") \
    .withColumn(
        "DATA_TEMP",
        to_date(concat(
            col("CO_ANO").cast("string"),
            lit("-"),
            lpad(col("CO_MES").cast("string"), 2, "0"),
            lit("-01")
        ), "yyyy-MM-dd")
    ) \
    .select(
        min("DATA_TEMP").alias("data_minima"),
        max("DATA_TEMP").alias("data_maxima")
    ).show()

# IMPORTAÇÃO - Select apenas das colunas necessárias + criação da data temporária
print("=== IMPORTAÇÃO ===")
df_import.select("CO_ANO", "CO_MES") \
    .withColumn(
        "DATA_TEMP",
        to_date(concat(
            col("CO_ANO").cast("string"),
            lit("-"),
            lpad(col("CO_MES").cast("string"), 2, "0"),
            lit("-01")
        ), "yyyy-MM-dd")
    ) \
    .select(
        min("DATA_TEMP").alias("data_minima"),
        max("DATA_TEMP").alias("data_maxima")
    ).show()

fim = time.time()
print(f"Tempo total: {fim - inicio:.3f} segundos")

=== EXPORTAÇÃO ===
+-----------+-----------+
|data_minima|data_maxima|
+-----------+-----------+
| 2010-01-01| 2025-06-01|
+-----------+-----------+

=== IMPORTAÇÃO ===
+-----------+-----------+
|data_minima|data_maxima|
+-----------+-----------+
| 2010-01-01| 2025-06-01|
+-----------+-----------+

Tempo total: 51.366 segundos


## Análises de missing values

In [ ]:
# Análise de Valores Ausentes (df Exportações)
print("\n--- Contagem de Valores Ausentes por Coluna ---")
for col_name in df_export.columns:
    null_count = df_export.filter(col(col_name).isNull()).count()
    print(f"Coluna '{col_name}': {null_count} valores ausentes")

# Análise de Valores Ausentes (df Importações)
print("\n--- Contagem de Valores Ausentes por Coluna ---")
for col_name in df_import.columns:
    null_count = df_import.filter(col(col_name).isNull()).count()
    print(f"Coluna '{col_name}': {null_count} valores ausentes")


--- Contagem de Valores Ausentes por Coluna ---
Coluna 'CO_ANO': 0 valores ausentes
Coluna 'CO_MES': 0 valores ausentes
Coluna 'SH4': 0 valores ausentes
Coluna 'CO_PAIS': 0 valores ausentes
Coluna 'SG_UF_MUN': 0 valores ausentes
Coluna 'CO_MUN': 0 valores ausentes
Coluna 'KG_LIQUIDO': 0 valores ausentes
Coluna 'VL_FOB': 0 valores ausentes

--- Contagem de Valores Ausentes por Coluna ---
Coluna 'CO_ANO': 0 valores ausentes
Coluna 'CO_MES': 0 valores ausentes
Coluna 'SH4': 0 valores ausentes
Coluna 'CO_PAIS': 0 valores ausentes
Coluna 'SG_UF_MUN': 0 valores ausentes
Coluna 'CO_MUN': 0 valores ausentes
Coluna 'KG_LIQUIDO': 0 valores ausentes
Coluna 'VL_FOB': 0 valores ausentes


## Estatísticas Descritivas

In [ ]:
# Análise Descritiva para Colunas Numéricas - EXPORTAÇÃO
print("\nEstatísticas Descritivas - EXPORTAÇÃO:")
df_export_stats = df_export.describe('KG_LIQUIDO', 'VL_FOB')

# Arredondar valores numéricos
df_export_stats_rounded = df_export_stats.select(
    "summary",
    round(col("KG_LIQUIDO"), 0).alias("peso_liquido_total_kg"),
    round(col("VL_FOB"), 0).alias("valor_fob_dolar_total")
)

df_export_stats_pd = df_export_stats_rounded.toPandas().T
print(df_export_stats_pd)

# Análise Descritiva para Colunas Numéricas - IMPORTAÇÃO
print("\nEstatísticas Descritivas - IMPORTAÇÃO:")
df_import_stats = df_import.describe('KG_LIQUIDO', 'VL_FOB')

# Arredondar valores numéricos
df_import_stats_rounded = df_import_stats.select(
    "summary",
    round(col("KG_LIQUIDO"), 2).alias("peso_liquido_total_kg"),
    round(col("VL_FOB"), 2).alias("valor_fob_dolar_total")
)

df_import_stats_pd = df_import_stats_rounded.toPandas().T
print(df_import_stats_pd)

# Limpeza de memória
del df_export_stats, df_export_stats_rounded, df_export_stats_pd
del df_import_stats, df_import_stats_rounded, df_import_stats_pd



Estatísticas Descritivas - EXPORTAÇÃO:
                                0         1           2    3              4
summary                     count      mean      stddev  min            max
peso_liquido_total_kg  13779191.0  743660.0  30904457.0  0.0  24030871000.0
valor_fob_dolar_total  13779191.0  279303.0   4419091.0  0.0   1733895757.0

Estatísticas Descritivas - IMPORTAÇÃO:
                                0          1           2    3             4
summary                     count       mean      stddev  min           max
peso_liquido_total_kg  22702125.0  106997.89  3208717.08  0.0  1823339652.0
valor_fob_dolar_total  22702125.0  142791.45  2393657.24  0.0  2661051787.0


## **Análise exploratória Econômica e Comercial**

### Série Histórica Anual das Exportações e Importações Brasileiras

In [ ]:
#===============================================================================
# Análise de Exportações e Importações por Ano
#===============================================================================

print("Análise comparativa: Exportações vs Importações (2010 - Jun/2025)")

# Dados de Exportação
exportacoes_por_ano = df_export.groupBy("CO_ANO").agg(
    round(sum("VL_FOB"), 2).alias("valor_fob_export"),
    round(sum("KG_LIQUIDO"), 2).alias("peso_export")
).orderBy("CO_ANO")

# Dados de Importação
importacoes_por_ano = df_import.groupBy("CO_ANO").agg(
    round(sum("VL_FOB"), 2).alias("valor_fob_import"),
    round(sum("KG_LIQUIDO"), 2).alias("peso_import")
).orderBy("CO_ANO")

# Converter para pandas
exp_ano_pd = exportacoes_por_ano.toPandas()
imp_ano_pd = importacoes_por_ano.toPandas()

# Merge dos dados por ano
dados_completos = exp_ano_pd.merge(imp_ano_pd, on='CO_ANO', how='outer').fillna(0)

#===============================================================================
# Dashboard Plotly Vertical (2x1) - Exportações vs Importações
#===============================================================================

# Criar subplots verticais (2 linhas, 1 coluna)
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        '<b>Valor FOB Total - Exportações vs Importações</b>',
        '<b>Peso Líquido Total - Exportações vs Importações</b>'
    ),
    vertical_spacing=0.20  # Mais espaço para acomodar legenda embaixo
)

# Função para formatar números
def formatar_valor(valor):
    if valor >= 1e9:
        return f"${valor/1e9:.1f}B"
    elif valor >= 1e6:
        return f"${valor/1e6:.1f}M"
    else:
        return f"${valor:,.0f}"

def formatar_peso(peso):
    if peso >= 1e9:
        return f"{peso/1e9:.1f}B kg"
    elif peso >= 1e6:
        return f"{peso/1e6:.1f}M kg"
    else:
        return f"{peso:,.0f} kg"

# 1. GRÁFICO SUPERIOR: Valores FOB
# Exportações
fig.add_trace(
    go.Scatter(
        x=dados_completos['CO_ANO'],
        y=dados_completos['valor_fob_export'],
        mode='lines+markers',
        name='Exportações',
        line=dict(color='darkgreen', width=3),
        marker=dict(size=8, color='darkgreen'),
        hovertemplate='<b>%{x}</b><br><b>Exportação:</b> %{text}<extra></extra>',
        text=[formatar_valor(v) for v in dados_completos['valor_fob_export']]
    ),
    row=1, col=1
)

# Importações
fig.add_trace(
    go.Scatter(
        x=dados_completos['CO_ANO'],
        y=dados_completos['valor_fob_import'],
        mode='lines+markers',
        name='Importações',
        line=dict(color='steelblue', width=3),
        marker=dict(size=8, color='steelblue', symbol='square'),
        hovertemplate='<b>%{x}</b><br><b>Importação:</b> %{text}<extra></extra>',
        text=[formatar_valor(v) for v in dados_completos['valor_fob_import']]
    ),
    row=1, col=1
)

# 2. GRÁFICO INFERIOR: Peso Líquido
# Exportações
fig.add_trace(
    go.Scatter(
        x=dados_completos['CO_ANO'],
        y=dados_completos['peso_export'],
        mode='lines+markers',
        name='Exportações (Peso)',
        line=dict(color='darkgreen', width=3),
        marker=dict(size=8, color='darkgreen'),
        hovertemplate='<b>%{x}</b><br><b>Exportação:</b> %{text}<extra></extra>',
        text=[formatar_peso(p) for p in dados_completos['peso_export']],
        showlegend=False  # Não mostrar na legenda (já está no gráfico superior)
    ),
    row=2, col=1
)

# Importações
fig.add_trace(
    go.Scatter(
        x=dados_completos['CO_ANO'],
        y=dados_completos['peso_import'],
        mode='lines+markers',
        name='Importações (Peso)',
        line=dict(color='steelblue', width=3),
        marker=dict(size=8, color='steelblue', symbol='square'),
        hovertemplate='<b>%{x}</b><br><b>Importação:</b> %{text}<extra></extra>',
        text=[formatar_peso(p) for p in dados_completos['peso_import']],
        showlegend=False  # Não mostrar na legenda (já está no gráfico superior)
    ),
    row=2, col=1
)

# Configurar layout com legenda na parte inferior
fig.update_layout(
    title={
        'text': '<b>Comércio Exterior Brasileiro</b> - Exportações vs Importações (2010 - Jun/2025)',
        'x': 0.5,
        'font': {'size': 20, 'family': 'Arial Black'}
    },
    height=900,  # Aumentar altura para acomodar legenda embaixo
    width=1200,
    font=dict(family="Arial", size=12),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
        orientation="h",          # Horizontal
        yanchor="top",           # Ancorar no topo da legenda
        y=-0.12,                 # Posição abaixo do gráfico (valor negativo)
        xanchor="center",        # Centralizar horizontalmente
        x=0.5,                   # Centro do gráfico
        bgcolor="rgba(255,255,255,0.9)",  # Fundo branco
        bordercolor="lightgray",
        borderwidth=1,
        font=dict(size=12)
    )
)

# Configurar eixos
fig.update_xaxes(title_text="<b>Ano</b>", showgrid=True, gridcolor='#E5E5E5')
fig.update_yaxes(title_text="<b>Valor FOB (USD)</b>", showgrid=True, gridcolor='#E5E5E5', row=1, col=1)
fig.update_yaxes(title_text="<b>Peso Líquido (kg)</b>", showgrid=True, gridcolor='#E5E5E5', row=2, col=1)

# Mostrar gráfico
fig.show()

# LIMPEZA DE MEMÓRIA
del exportacoes_por_ano, importacoes_por_ano, exp_ano_pd, imp_ano_pd, dados_completos


Análise comparativa: Exportações vs Importações (2010 - Jun/2025)


### Série Histórica Mensal das Exportações e Importações Brasileiras

In [ ]:
#===============================================================================
# Análise de Exportações E Importações por Mês
#===============================================================================

print("Análise comparativa mensal: Exportações vs Importações (Jan/2010 - Jun/2025)")

# Dados de Exportação por mês
exportacoes_por_mes = df_export.groupBy("CO_ANO", "CO_MES").agg(
    round(sum("VL_FOB"), 2).alias("valor_fob_export"),
    round(sum("KG_LIQUIDO"), 2).alias("peso_export")
).orderBy("CO_ANO", "CO_MES")

# Dados de Importação por mês
importacoes_por_mes = df_import.groupBy("CO_ANO", "CO_MES").agg(
    round(sum("VL_FOB"), 2).alias("valor_fob_import"),
    round(sum("KG_LIQUIDO"), 2).alias("peso_import")
).orderBy("CO_ANO", "CO_MES")

# Converter para pandas
exp_mes_pd = exportacoes_por_mes.toPandas()
imp_mes_pd = importacoes_por_mes.toPandas()

# Criar coluna de data para melhor visualização
exp_mes_pd['data'] = exp_mes_pd['CO_ANO'].astype(str) + '-' + exp_mes_pd['CO_MES'].astype(str).str.zfill(2)
imp_mes_pd['data'] = imp_mes_pd['CO_ANO'].astype(str) + '-' + imp_mes_pd['CO_MES'].astype(str).str.zfill(2)

# Converter para datetime
exp_mes_pd['data'] = pd.to_datetime(exp_mes_pd['data'], format='%Y-%m')
imp_mes_pd['data'] = pd.to_datetime(imp_mes_pd['data'], format='%Y-%m')

# Merge dos dados por mês
dados_completos = exp_mes_pd.merge(imp_mes_pd, on=['CO_ANO', 'CO_MES', 'data'], how='outer').fillna(0)
dados_completos = dados_completos.sort_values('data')

print(f"Período: {dados_completos['data'].min()} até {dados_completos['data'].max()}")

#===============================================================================
# Dashboard Plotly Vertical (2x1) - Série Temporal Mensal
#===============================================================================

# Criar subplots verticais (2 linhas, 1 coluna)
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        '<b>Valor FOB Total Mensal - Exportações vs Importações</b>',
        '<b>Peso Líquido Total Mensal - Exportações vs Importações</b>'
    ),
    vertical_spacing=0.20
)

# Função para formatar números
def formatar_valor(valor):
    if valor >= 1e9:
        return f"${valor/1e9:.1f}B"
    elif valor >= 1e6:
        return f"${valor/1e6:.1f}M"
    else:
        return f"${valor:,.0f}"

def formatar_peso(peso):
    if peso >= 1e9:
        return f"{peso/1e9:.1f}B kg"
    elif peso >= 1e6:
        return f"{peso/1e6:.1f}M kg"
    else:
        return f"{peso:,.0f} kg"

# 1. GRÁFICO SUPERIOR: Valores FOB
# Exportações
fig.add_trace(
    go.Scatter(
        x=dados_completos['data'],
        y=dados_completos['valor_fob_export'],
        mode='lines',  # Sem markers para dados mensais (muitos pontos)
        name='Exportações',
        line=dict(color='darkgreen', width=2),
        hovertemplate='<b>%{x|%Y-%m}</b><br><b>Exportação:</b> %{text}<extra></extra>',
        text=[formatar_valor(v) for v in dados_completos['valor_fob_export']]
    ),
    row=1, col=1
)

# Importações
fig.add_trace(
    go.Scatter(
        x=dados_completos['data'],
        y=dados_completos['valor_fob_import'],
        mode='lines',
        name='Importações',
        line=dict(color='steelblue', width=2),
        hovertemplate='<b>%{x|%Y-%m}</b><br><b>Importação:</b> %{text}<extra></extra>',
        text=[formatar_valor(v) for v in dados_completos['valor_fob_import']]
    ),
    row=1, col=1
)

# 2. GRÁFICO INFERIOR: Peso Líquido
# Exportações
fig.add_trace(
    go.Scatter(
        x=dados_completos['data'],
        y=dados_completos['peso_export'],
        mode='lines',
        name='Exportações (Peso)',
        line=dict(color='darkgreen', width=2),
        hovertemplate='<b>%{x|%Y-%m}</b><br><b>Exportação:</b> %{text}<extra></extra>',
        text=[formatar_peso(p) for p in dados_completos['peso_export']],
        showlegend=False
    ),
    row=2, col=1
)

# Importações
fig.add_trace(
    go.Scatter(
        x=dados_completos['data'],
        y=dados_completos['peso_import'],
        mode='lines',
        name='Importações (Peso)',
        line=dict(color='steelblue', width=2),
        hovertemplate='<b>%{x|%Y-%m}</b><br><b>Importação:</b> %{text}<extra></extra>',
        text=[formatar_peso(p) for p in dados_completos['peso_import']],
        showlegend=False
    ),
    row=2, col=1
)

# Configurar layout
fig.update_layout(
    title={
        'text': '<b>Comércio Exterior Brasileiro</b> - Série Temporal Mensal (Jan/2010 - Jun/2025)',
        'x': 0.5,
        'font': {'size': 20, 'family': 'Arial Black'}
    },
    height=900,
    width=1200,  # Mais largo para dados mensais
    font=dict(family="Arial", size=12),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.12,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(255,255,255,0.9)",
        bordercolor="lightgray",
        borderwidth=1,
        font=dict(size=12)
    )
)

# Configurar eixos
fig.update_xaxes(
    title_text="<b>Período</b>",
    showgrid=True,
    gridcolor='#E5E5E5',
    tickformat='%Y-%m'  # Formato de data
)
fig.update_yaxes(title_text="<b>Valor FOB (USD)</b>", showgrid=True, gridcolor='#E5E5E5', row=1, col=1)
fig.update_yaxes(title_text="<b>Peso Líquido (kg)</b>", showgrid=True, gridcolor='#E5E5E5', row=2, col=1)

# Mostrar gráfico
fig.show()

# LIMPEZA DE MEMÓRIA
del exportacoes_por_mes, importacoes_por_mes, exp_mes_pd, imp_mes_pd, dados_completos


Análise comparativa mensal: Exportações vs Importações (Jan/2010 - Jun/2025)
Período: 2010-01-01 00:00:00 até 2025-06-01 00:00:00


In [ ]:
#===============================================================================
# Análise de Exportações E Importações por Mês
#===============================================================================

print("Análise comparativa mensal: Exportações vs Importações (Jan/2010 - Jun/2025)")

# Dados de Exportação por mês
exportacoes_por_mes = df_export.groupBy("CO_ANO", "CO_MES").agg(
    round(sum("VL_FOB"), 2).alias("valor_fob_export"),
    round(sum("KG_LIQUIDO"), 2).alias("peso_export")
).orderBy("CO_ANO", "CO_MES")

# Dados de Importação por mês
importacoes_por_mes = df_import.groupBy("CO_ANO", "CO_MES").agg(
    round(sum("VL_FOB"), 2).alias("valor_fob_import"),
    round(sum("KG_LIQUIDO"), 2).alias("peso_import")
).orderBy("CO_ANO", "CO_MES")

# Converter para pandas
exp_mes_pd = exportacoes_por_mes.toPandas()
imp_mes_pd = importacoes_por_mes.toPandas()

# Criar coluna de data para melhor visualização
exp_mes_pd['data'] = exp_mes_pd['CO_ANO'].astype(str) + '-' + exp_mes_pd['CO_MES'].astype(str).str.zfill(2)
imp_mes_pd['data'] = imp_mes_pd['CO_ANO'].astype(str) + '-' + imp_mes_pd['CO_MES'].astype(str).str.zfill(2)

# Converter para datetime
exp_mes_pd['data'] = pd.to_datetime(exp_mes_pd['data'], format='%Y-%m')
imp_mes_pd['data'] = pd.to_datetime(imp_mes_pd['data'], format='%Y-%m')

# Merge dos dados por mês
dados_completos = exp_mes_pd.merge(imp_mes_pd, on=['CO_ANO', 'CO_MES', 'data'], how='outer').fillna(0)
dados_completos = dados_completos.sort_values('data')

#===============================================================================
# Plot Plotly
#===============================================================================

# Calcular saldo comercial
dados_completos['saldo_comercial'] = dados_completos['valor_fob_export'] - dados_completos['valor_fob_import']

# Separar dados de superávit e déficit
superavit = dados_completos[dados_completos['saldo_comercial'] >= 0]
deficit = dados_completos[dados_completos['saldo_comercial'] < 0]

# Criar figura
fig = go.Figure()

# Superávit
fig.add_trace(
    go.Bar(
        x=superavit['data'],
        y=superavit['saldo_comercial'],
        name='Superávit',
        marker_color='green',
        hovertemplate='<b>%{x|%Y-%m}</b><br><b>Saldo:</b> %{y:,.0f} USD<extra></extra>'
    )
)

# Déficit
fig.add_trace(
    go.Bar(
        x=deficit['data'],
        y=deficit['saldo_comercial'],
        name='Déficit',
        marker_color='red',
        hovertemplate='<b>%{x|%Y-%m}</b><br><b>Saldo:</b> %{y:,.0f} USD<extra></extra>'
    )
)

# Layout e legenda
fig.update_layout(
    title='<b>Saldo da Balança Comercial Brasileira (Mensal)</b><br>(Exportações - Importações)',
    xaxis_title='Período',
    yaxis_title='Saldo (USD)',
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500,
    width=1000,
    font=dict(family="Arial", size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5,
        bgcolor="rgba(255,255,255,0.9)",
        bordercolor="lightgray",
        borderwidth=1
    )
)

fig.update_xaxes(showgrid=True, gridcolor='lightgray', tickformat='%Y-%m')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()

# LIMPEZA DE MEMÓRIA
del exportacoes_por_mes, importacoes_por_mes, exp_mes_pd, imp_mes_pd, dados_completos


Análise comparativa mensal: Exportações vs Importações (Jan/2010 - Jun/2025)


### **Top 10 principais destinos das exportações brasileiras**

In [ ]:
#===============================================================================
# Top 10 Exportações e Importações - Gráficos Separados
#===============================================================================

from pyspark.sql.functions import col, sum, countDistinct, desc
import plotly.graph_objects as go

# Importar dicionário de países
df_paises_master = spark.read.parquet(f"{base_path}df_paises_master.parquet")

# Definir período: Junho/2024 a Junho/2025
print("Análise: Jun/2024 - Jun/2025")
filtro_periodo = (
    ((col("CO_ANO") == 2024) & (col("CO_MES") >= 6)) |
    ((col("CO_ANO") == 2025) & (col("CO_MES") <= 6))
)

#===================== Exportações =====================#
top_destinos_export = (
    df_export.filter(filtro_periodo)
    .groupBy("CO_PAIS")
    .agg(
        sum("VL_FOB").alias("valor_total"),
        sum("KG_LIQUIDO").alias("peso_total"),
        countDistinct("CO_MUN").alias("num_municipios")
    )
    .join(df_paises_master, "CO_PAIS", "left")
    .select("NO_PAIS", "valor_total", "peso_total", "num_municipios")
    .orderBy(desc("valor_total"))
    .limit(10)
)

#===================== Importações =====================#
top_destinos_import = (
    df_import.filter(filtro_periodo)
    .groupBy("CO_PAIS")
    .agg(
        sum("VL_FOB").alias("valor_total"),
        sum("KG_LIQUIDO").alias("peso_total"),
        countDistinct("CO_MUN").alias("num_municipios")
    )
    .join(df_paises_master, "CO_PAIS", "left")
    .select("NO_PAIS", "valor_total", "peso_total", "num_municipios")
    .orderBy(desc("valor_total"))
    .limit(10)
)

#===================== Conversão para Pandas =====================#
top_export_pd = top_destinos_export.toPandas()
top_import_pd = top_destinos_import.toPandas()

#===================== Função Formatadora =====================#
def formatar_valor(valor):
    if valor >= 1e9:
        return f"${valor/1e9:.2f}B"
    elif valor >= 1e6:
        return f"${valor/1e6:.2f}M"
    else:
        return f"${valor:,.2f}"

#===================== Função de Plotagem =====================#
def plot_top_destinos(df, titulo, cor):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            x=df['NO_PAIS'],
            y=df['valor_total'],
            name='Valor FOB Total',
            marker=dict(color=cor, line=dict(color='white', width=1)),
            hovertemplate='<b>%{x}</b><br>' +
                          '<b>Valor FOB:</b> %{text}<br>' +
                          '<b>Municípios:</b> %{customdata}<extra></extra>',
            text=[formatar_valor(v) for v in df['valor_total']],
            customdata=df['num_municipios'],
            textposition='outside',
            textfont=dict(size=11, color='black')
        )
    )

    fig.update_layout(
        title={
            'text': f'<b>{titulo}</b><br><span style="font-size:14px">Junho/2024 - Junho/2025</span>',
            'x': 0.5,
            'font': {'size': 20, 'family': 'Arial Black'}
        },
        width=1100,
        height=600,
        font=dict(family="Arial", size=12),
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=False
    )

    fig.update_xaxes(
        title_text="<b>País</b>",
        showgrid=False,
        tickangle=45,
        title_font=dict(size=14, family='Arial Black')
    )

    fig.update_yaxes(
        title_text="<b>Valor FOB Total (USD)</b>",
        showgrid=True,
        gridcolor='#E5E5E5',
        title_font=dict(size=14, family='Arial Black'),
        tickformat='.0s'
    )

    fig.show()

#===================== Gerar Gráficos =====================#
plot_top_destinos(top_export_pd, "Top 10 Destinos das Exportações Brasileiras", "darkgreen")
plot_top_destinos(top_import_pd, "Top 10 Origem das Importações Brasileiras", "steelblue")

#===================== Estatísticas =====================#
print(f"Total Exportado (Top 10): ${top_export_pd['valor_total'].sum() / 1e9:.2f}B")
print(f"Total Importado (Top 10): ${top_import_pd['valor_total'].sum() / 1e9:.2f}B")

#===================== Limpeza =====================#
del top_destinos_export, top_destinos_import


Análise: Jun/2024 - Jun/2025


Total Exportado (Top 10): $242.06B
Total Importado (Top 10): $216.55B


### Gráfico Top 10 Municípios com maiores Exportações e Importações

In [ ]:
#===============================================================================
# Função auxiliar para processar exportações ou importações
#===============================================================================
df_geo_master = spark.read.parquet(f"{base_path}df_geo_master.parquet")

def processar_top_municipios(df_comex, df_geo_master, tipo='export'):
    df_geo = df_comex.join(
        df_geo_master,
        df_comex.CO_MUN == df_geo_master.CO_MUN_GEO,
        "left"
    )

    df_final = df_geo.select(
        col("VL_FOB").alias("valor_fob_dolar"),
        col("KG_LIQUIDO").alias("peso_liquido_kg"),
        col("NO_MUN_MIN").alias("nome_municipio"),
        col("SG_UF").alias("sigla_uf"),
        col("CO_ANO")
    )

    return (
        df_final.groupBy("nome_municipio", "sigla_uf")
        .agg(
            round(sum("valor_fob_dolar"), 2).alias("valor_fob_total_dolar_municipio"),
            round(sum("peso_liquido_kg"), 2).alias("peso_total_kg_municipio"),
            countDistinct("CO_ANO").alias("num_anos_ativo")
        )
        .orderBy(desc("valor_fob_total_dolar_municipio"))
        .limit(10)
    )

#===============================================================================
# Gerar dataframes para plotagem
#===============================================================================
top_municipios_export = processar_top_municipios(df_export, df_geo_master, tipo='export')
top_municipios_import = processar_top_municipios(df_import, df_geo_master, tipo='import')

# Converter para pandas
top_export_pd = top_municipios_export.toPandas()
top_import_pd = top_municipios_import.toPandas()

# Criar coluna com Município - UF
top_export_pd['municipio_uf'] = top_export_pd['nome_municipio'] + ' - ' + top_export_pd['sigla_uf']
top_import_pd['municipio_uf'] = top_import_pd['nome_municipio'] + ' - ' + top_import_pd['sigla_uf']

#===============================================================================
# Função formatadora
#===============================================================================
def formatar_valor(valor):
    if valor >= 1e9:
        return f"${valor/1e9:.1f}B"
    elif valor >= 1e6:
        return f"${valor/1e6:.1f}M"
    else:
        return f"${valor:,.0f}"

#===============================================================================
# Criação dos subplots
#===============================================================================
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=[
        '<b>Top 10 Municípios Exportadores - Valor FOB Total</b>',
        '<b>Top 10 Municípios Importadores - Valor FOB Total</b>'
    ],
    vertical_spacing=0.25
)

# Exportações
fig.add_trace(
    go.Bar(
        x=top_export_pd['municipio_uf'],  # <-- CORREÇÃO FEITA AQUI
        y=top_export_pd['valor_fob_total_dolar_municipio'],
        name='Exportações',
        marker=dict(color='darkgreen'),
        hovertemplate='<b>%{x}</b><br><b>Valor FOB:</b> %{text}<br><b>Anos Ativos:</b> %{customdata}<extra></extra>',
        text=[formatar_valor(v) for v in top_export_pd['valor_fob_total_dolar_municipio']],
        customdata=top_export_pd['num_anos_ativo'],
        textposition='outside'
    ),
    row=1, col=1
)

# Importações
fig.add_trace(
    go.Bar(
        x=top_import_pd['municipio_uf'],
        y=top_import_pd['valor_fob_total_dolar_municipio'],
        name='Importações',
        marker=dict(color='steelblue'),
        hovertemplate='<b>%{x}</b><br><b>Valor FOB:</b> %{text}<br><b>Anos Ativos:</b> %{customdata}<extra></extra>',
        text=[formatar_valor(v) for v in top_import_pd['valor_fob_total_dolar_municipio']],
        customdata=top_import_pd['num_anos_ativo'],
        textposition='outside'
    ),
    row=2, col=1
)

#===============================================================================
# Layout do gráfico
#===============================================================================
fig.update_layout(
    title={
        'text': '<b>Top 10 Municípios no Comércio Exterior Brasileiro</b><br>'
                '<span style="font-size:14px">Período: 2010 - Jun/2025</span>',
        'x': 0.5,
        'font': {'size': 20, 'family': 'Arial Black'}
    },
    height=1000,
    width=1500,
    font=dict(family="Arial", size=13),
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=False
)

fig.update_xaxes(
    title_text="<b>Município - UF</b>",
    showgrid=False,
    tickangle=45
)
fig.update_yaxes(
    title_text="<b>Valor FOB (USD)</b>",
    showgrid=True,
    gridcolor='#E5E5E5'
)

fig.show()

#===============================================================================
# LIMPEZA DE MEMÓRIA
#===============================================================================
del top_export_pd, top_import_pd, top_municipios_export, top_municipios_import


### Top 10 Países no Comércio Exterior Brasileiro

In [ ]:
#===============================================================================
# Filtro de Período e Análise de Exportações e Importações por País
#===============================================================================

print("Análise: Top 10 Países - Jun/2024 a Jun/2025")

# Filtro do período
filtro_periodo = (
    ((col("CO_ANO") == 2024) & (col("CO_MES") >= 6)) |
    ((col("CO_ANO") == 2025) & (col("CO_MES") <= 6))
)

#===============================================================================
# EXPORTAÇÕES
#===============================================================================

# Join exportações com dicionário de países (com filtro de período)
df_export_com_pais = df_export.filter(filtro_periodo).join(
    df_paises_master,
    df_export.CO_PAIS == df_paises_master.CO_PAIS,
    "left"
)

df_export_pais_plt = df_export_com_pais.select(
    "CO_ANO",
    "CO_MES",
    col("VL_FOB").alias("valor_fob_dolar"),
    col("KG_LIQUIDO").alias("peso_liquido_kg"),
    col("NO_PAIS").alias("nome_pais"),
    "NO_BLOCO"
)

top_export_pais_plt = df_export_pais_plt.groupBy("nome_pais", "NO_BLOCO").agg(
    round(sum("valor_fob_dolar"), 2).alias("valor_fob_total_dolar"),
    round(sum("peso_liquido_kg"), 2).alias("peso_total_kg"),
    countDistinct("CO_ANO").alias("num_anos_ativo"),
    countDistinct("CO_MES").alias("num_meses_ativo")
).orderBy(desc("valor_fob_total_dolar")).limit(10)

#===============================================================================
# IMPORTAÇÕES
#===============================================================================

# Join importações com dicionário de países (com filtro de período)
df_import_com_pais = df_import.filter(filtro_periodo).join(
    df_paises_master,
    df_import.CO_PAIS == df_paises_master.CO_PAIS,
    "left"
)

df_import_pais_plt = df_import_com_pais.select(
    "CO_ANO",
    "CO_MES",
    col("VL_FOB").alias("valor_fob_dolar"),
    col("KG_LIQUIDO").alias("peso_liquido_kg"),
    col("NO_PAIS").alias("nome_pais"),
    "NO_BLOCO"
)

top_import_pais_plt = df_import_pais_plt.groupBy("nome_pais", "NO_BLOCO").agg(
    round(sum("valor_fob_dolar"), 2).alias("valor_fob_total_dolar"),
    round(sum("peso_liquido_kg"), 2).alias("peso_total_kg"),
    countDistinct("CO_ANO").alias("num_anos_ativo"),
    countDistinct("CO_MES").alias("num_meses_ativo")
).orderBy(desc("valor_fob_total_dolar")).limit(10)

#===============================================================================
# Plotagem com Plotly - Dashboard Vertical
#===============================================================================

# Converter para pandas
top_export_pd = top_export_pais_plt.toPandas()
top_import_pd = top_import_pais_plt.toPandas()

# Função para formatar valores
def formatar_valor(valor):
    if valor >= 1e9:
        return f"${valor/1e9:.1f}B"
    elif valor >= 1e6:
        return f"${valor/1e6:.1f}M"
    else:
        return f"${valor:,.0f}"

# Criar subplots verticais
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        '<b>Top 10 Países - Exportações Brasileiras</b>',
        '<b>Top 10 Países - Importações Brasileiras</b>'
    ),
    vertical_spacing=0.20
)

# 1. GRÁFICO SUPERIOR: Exportações
fig.add_trace(
    go.Bar(
        x=top_export_pd['nome_pais'],
        y=top_export_pd['valor_fob_total_dolar'],
        name='Exportações',
        marker=dict(color='darkgreen'),
        hovertemplate='<b>%{x}</b><br>' +
                      '<b>Valor FOB:</b> %{text}<br>' +
                      '<b>Bloco:</b> %{customdata}<br>' +
                      '<extra></extra>',
        text=[formatar_valor(v) for v in top_export_pd['valor_fob_total_dolar']],
        customdata=top_export_pd['NO_BLOCO'],
        textposition='outside',
        textfont=dict(size=15, color='black')
    ),
    row=1, col=1
)

# 2. GRÁFICO INFERIOR: Importações
fig.add_trace(
    go.Bar(
        x=top_import_pd['nome_pais'],
        y=top_import_pd['valor_fob_total_dolar'],
        name='Importações',
        marker=dict(color='steelblue'),
        hovertemplate='<b>%{x}</b><br>' +
                      '<b>Valor FOB:</b> %{text}<br>' +
                      '<b>Bloco:</b> %{customdata}<br>' +
                      '<extra></extra>',
        text=[formatar_valor(v) for v in top_import_pd['valor_fob_total_dolar']],
        customdata=top_import_pd['NO_BLOCO'],
        textposition='outside',
        textfont=dict(size=15, color='black')
    ),
    row=2, col=1
)

# Configurar layout
fig.update_layout(
    title={
        'text': '<b>Top 10 Países no Comércio Exterior Brasileiro</b><br>' +
                '<span style="font-size:14px">Período: Junho/2024 - Junho/2025</span>',
        'x': 0.5,
        'font': {'size': 20, 'family': 'Arial Black'}
    },
    height=1000,
    width=1500,
    font=dict(family="Arial", size=15),
    plot_bgcolor='white',
    paper_bgcolor='white',
    showlegend=False
)

# Configurar eixos
fig.update_xaxes(
    title_text="<b>País</b>",
    showgrid=False,
    tickangle=45
)

fig.update_yaxes(
    title_text="<b>Valor FOB (USD)</b>",
    showgrid=True,
    gridcolor='#E5E5E5'
)

# Mostrar gráfico
fig.show()

# LIMPEZA DE MEMÓRIA
del df_export_com_pais, df_import_com_pais, df_export_pais_plt, df_import_pais_plt, top_export_pais_plt, top_import_pais_plt, top_export_pd, top_import_pd


Análise: Top 10 Países - Jun/2024 a Jun/2025


### Top 10 Produtos no Comércio Exterior Brasileiro

## **Participação dos EUA nas Exportações Brasileiras por Produto**

### NOTA: Realizei algumas agregações para esse exercicio, que não estão em linha com as agregações padrões


In [ ]:
# 1. Carregar o dicionário de agregação
df_ncm_silvio = spark.read.parquet(f"{base_path}Minha_agregacao.parquet")

# 2. Criar coluna id_sh4
df_ncm_silvio = df_ncm_silvio.withColumn(
    "id_sh4",
    substring(regexp_replace(col("id_sh6").cast("string"), "\\.0$", ""), 1, 4).cast("int")
)

# 3. Join com dados de exportação + filtro de período
df_join = df_export.join(
    df_ncm_silvio,
    df_export.SH4 == df_ncm_silvio.id_sh4,
    "left"
).filter(
    ((col("CO_ANO") == 2024) & (col("CO_MES") >= 7)) |
    ((col("CO_ANO") == 2025) & (col("CO_MES") <= 6))
).select(
    "categoria_agregada",
    "CO_PAIS",
    "VL_FOB",
    "KG_LIQUIDO"
)

# 4. Agregar total exportado por produto e por país
df_pais = df_join.groupBy("categoria_agregada", "CO_PAIS").agg(
    sum("VL_FOB").alias("vl_fob_pais"),
    sum("KG_LIQUIDO").alias("qtd_kgl_pais")
)

# 5. Agregar total exportado por produto (todos os países)
df_total = df_join.groupBy("categoria_agregada").agg(
    sum("VL_FOB").alias("vl_fob_total"),
    sum("KG_LIQUIDO").alias("qtd_kgl_total")
)

# 6. Juntar os dois e calcular a participação percentual
df_participacao = df_pais.join(
    df_total, on="categoria_agregada"
).withColumn(
    "perc_participacao_valor", round(100 * col("vl_fob_pais") / col("vl_fob_total"), 1)
).withColumn(
    "perc_participacao_qtd", round(100 * col("qtd_kgl_pais") / col("qtd_kgl_total"), 1)
)

# 7. Filtrar só os EUA (código país = 249 no COMEX) ou China (160)

# Para EUA:
df_eua_valor = df_participacao.filter(col("CO_PAIS") == 249).select(
    "categoria_agregada", "perc_participacao_valor"
).orderBy(col("perc_participacao_valor").desc())

df_eua_qtd = df_participacao.filter(col("CO_PAIS") == 249).select(
    "categoria_agregada", "perc_participacao_qtd"
).orderBy(col("perc_participacao_qtd").desc())


# ========================================
# CONVERTENDO PARA PANDAS E CRIANDO GRÁFICOS
# ========================================

# 8. Converter para Pandas (Top 20)
df_valor_pd = df_eua_valor.toPandas().head(20)
df_qtd_pd = df_eua_qtd.toPandas().head(20)

# 9. Gráfico por Valor com cor escura (verde escuro)
fig_valor = go.Figure(data=go.Bar(
    x=df_valor_pd['perc_participacao_valor'],
    y=df_valor_pd['categoria_agregada'],
    orientation='h',
    marker=dict(color='darkgreen'),
    text=df_valor_pd['perc_participacao_valor'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_valor.update_layout(
    title='Top 20 Produtos - Participação dos EUA nas Exportações por Valor (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_valor.show()

# 10. Gráfico por Quantidade com cor azul (steelblue)
fig_qtd = go.Figure(data=go.Bar(
    x=df_qtd_pd['perc_participacao_qtd'],
    y=df_qtd_pd['categoria_agregada'],
    orientation='h',
    marker=dict(color='steelblue'),
    text=df_qtd_pd['perc_participacao_qtd'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_qtd.update_layout(
    title='Top 20 Produtos - Participação dos EUA nas Exportações por Quantidade (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_qtd.show()

# LIMPEZA DE MEMÓRIA
del df_join, df_pais, df_total, df_participacao, df_eua_valor, df_eua_qtd, df_valor_pd, df_qtd_pd, fig_valor, fig_qtd


**Interpretação dos Dados:**

Cada percentual mostra quanto das exportações brasileiras de cada produto tem os EUA como destino.

Exemplos olhando para o valor FOB das exportações:

* Veículos Aéreos e Peças: 66,8% → Dos 100% exportados pelo Brasil, 66,8% vai para os EUA.


**Por que não soma 100%?**

Os percentuais não devem somar 100% porque cada produto tem seu próprio "universo" de exportações. Estamos medindo a dependência dos EUA para cada categoria de produto, não a composição da pauta exportadora.

**Como interpretar:**

* Percentuais altos (>40%): Produtos com forte dependência do mercado americano
* Percentuais médios (20-40%): Mercado americano relevante, mas diversificado
* Percentuais baixos (<20%): Pouca dependência dos EUA, mercados bem diversificados

____________

## **Participação da China nas Exportações Brasileiras por Produto**

### NOTA: Realizei algumas agregações para esse exercicio, que não estão em linha com as agregações padrões

In [ ]:

# ========================================
# CRIANDO ANÁLISE PARA CHINA
# ========================================

# 1. Carregar o dicionário de agregação
df_ncm_silvio = spark.read.parquet(f"{base_path}Minha_agregacao.parquet")

# 2. Criar coluna id_sh4
df_ncm_silvio = df_ncm_silvio.withColumn(
    "id_sh4",
    substring(regexp_replace(col("id_sh6").cast("string"), "\\.0$", ""), 1, 4).cast("int")
)

# 3. Join com dados de exportação + filtro de período
df_join = df_export.join(
    df_ncm_silvio,
    df_export.SH4 == df_ncm_silvio.id_sh4,
    "left"
).filter(
    ((col("CO_ANO") == 2024) & (col("CO_MES") >= 7)) |
    ((col("CO_ANO") == 2025) & (col("CO_MES") <= 6))
).select(
    "categoria_agregada",
    "CO_PAIS",
    "VL_FOB",
    "KG_LIQUIDO"
)

# 4. Agregar total exportado por produto e por país
df_pais = df_join.groupBy("categoria_agregada", "CO_PAIS").agg(
    sum("VL_FOB").alias("vl_fob_pais"),
    sum("KG_LIQUIDO").alias("qtd_kgl_pais")
)

# 5. Agregar total exportado por produto (todos os países)
df_total = df_join.groupBy("categoria_agregada").agg(
    sum("VL_FOB").alias("vl_fob_total"),
    sum("KG_LIQUIDO").alias("qtd_kgl_total")
)

# 6. Juntar os dois e calcular a participação percentual
df_participacao = df_pais.join(
    df_total, on="categoria_agregada"
).withColumn(
    "perc_participacao_valor", round(100 * col("vl_fob_pais") / col("vl_fob_total"), 1)
).withColumn(
    "perc_participacao_qtd", round(100 * col("qtd_kgl_pais") / col("qtd_kgl_total"), 1)
)

# 7. Filtrar só a CHINA (código país = 160 no COMEX)
df_china_valor = df_participacao.filter(col("CO_PAIS") == 160).select(
    "categoria_agregada", "perc_participacao_valor"
).orderBy(col("perc_participacao_valor").desc())

df_china_qtd = df_participacao.filter(col("CO_PAIS") == 160).select(
    "categoria_agregada", "perc_participacao_qtd"
).orderBy(col("perc_participacao_qtd").desc())

# ========================================
# CONVERTENDO PARA PANDAS E CRIANDO GRÁFICOS
# ========================================

# 8. Converter para Pandas (Top 20)
df_valor_pd = df_china_valor.toPandas().head(20)
df_qtd_pd = df_china_qtd.toPandas().head(20)

# 9. Gráfico por Valor com cor escura (verde escuro)
fig_valor = go.Figure(data=go.Bar(
    x=df_valor_pd['perc_participacao_valor'],
    y=df_valor_pd['categoria_agregada'],
    orientation='h',
    marker=dict(color='darkgreen'),
    text=df_valor_pd['perc_participacao_valor'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_valor.update_layout(
    title='Top 20 Produtos - Participação da China nas Exportações por Valor (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_valor.show()

# 10. Gráfico por Quantidade com cor azul (steelblue)
fig_qtd = go.Figure(data=go.Bar(
    x=df_qtd_pd['perc_participacao_qtd'],
    y=df_qtd_pd['categoria_agregada'],
    orientation='h',
    marker=dict(color='steelblue'),
    text=df_qtd_pd['perc_participacao_qtd'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_qtd.update_layout(
    title='Top 20 Produtos - Participação da China nas Exportações por Quantidade (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_qtd.show()

# LIMPEZA DE MEMÓRIA
del df_join, df_pais, df_total, df_participacao, df_china_valor, df_china_qtd, df_valor_pd, df_qtd_pd, fig_valor, fig_qtd


**Interpretação dos Dados:**

Cada percentual mostra quanto das exportações brasileiras de cada produto tem a China como destino.

Exemplos olhando para o valor FOB das exportações:

* Metais de Terras Raras: 75.6% → Dos 100% exportado pelo Brasil, 75,6% vai para a China.


**Por que não soma 100%?**

Os percentuais não devem somar 100% porque cada produto tem seu próprio "universo" de exportações. Estamos medindo a dependência da China para cada categoria de produto, não a composição da pauta exportadora.

**Como interpretar:**

* Percentuais altos (>40%): Produtos com forte dependência do mercado americano
* Percentuais médios (20-40%): Mercado americano relevante, mas diversificado
* Percentuais baixos (<20%): Pouca dependência da China, mercados bem diversificados

___________

## Participação da Rússia nas Exportações Brasileiras por Produto

### NOTA: Realizei algumas agregações para esse exercicio, que não estão em linha com as agregações padrões

In [ ]:
df_produtoscuci_master_select = spark.read.parquet(f"{base_path}df_produtoscuci_master_select.parquet")
df_produtoscuci_master_select.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|CO_SH4|          NO_NCM_POR|       NO_CUCI_GRUPO|     NO_CUCI_DIVISAO|          NO_PPE_MIN|          NO_CGCE_N2|          NO_CGCE_N1|        NO_FAT_AGREG|CO_CUCI_GRUPO|       NO_ISIC_SECAO|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+
|  4901|Livros, brochuras...|   Material impresso|Artigos manufatur...|Livros, revistas,...|Bens de consumo s...|BENS DE CONSUMO (BC)|PRODUTOS MANUFATU...|          892|     Outros Produtos|
|  4901|Dicionários e enc...|   Material impresso|Artigos manufatur...|Livros, revistas,...|Bens de consumo s...|BENS DE CONSUMO (BC)|PRODUTOS MANUFATU...|          892|     Outros Produtos|
|  4901|Outros livros, br...|   Material impr

In [ ]:

# 1. Carregar o dicionário de agregação
df_ncm = spark.read.parquet(f"{base_path}df_produtoscuci_master_select.parquet")

# 3. Join com dados de exportação + filtro de período
df_join = df_export.join(
    df_ncm,
    df_export.SH4 == df_ncm.CO_SH4,
    "left"
).filter(
    ((col("CO_ANO") == 2024) & (col("CO_MES") >= 7)) |
    ((col("CO_ANO") == 2025) & (col("CO_MES") <= 6))
).select(
    "NO_CUCI_DIVISAO",
    "CO_PAIS",
    "VL_FOB",
    "KG_LIQUIDO"
)

# 4. Agregar total exportado por produto e por país
df_pais = df_join.groupBy("NO_CUCI_DIVISAO", "CO_PAIS").agg(
    sum("VL_FOB").alias("vl_fob_pais"),
    sum("KG_LIQUIDO").alias("qtd_kgl_pais")
)

# 5. Agregar total exportado por produto (todos os países)
df_total = df_join.groupBy("NO_CUCI_DIVISAO").agg(
    sum("VL_FOB").alias("vl_fob_total"),
    sum("KG_LIQUIDO").alias("qtd_kgl_total")
)

# 6. Juntar os dois e calcular a participação percentual
df_participacao = df_pais.join(
    df_total, on="NO_CUCI_DIVISAO"
).withColumn(
    "perc_participacao_valor", round(100 * col("vl_fob_pais") / col("vl_fob_total"), 1)
).withColumn(
    "perc_participacao_qtd", round(100 * col("qtd_kgl_pais") / col("qtd_kgl_total"), 1)
)

# 7. Filtrar só os Russia (código país = 676 no COMEX)

# Para Russia:
df_Russia_valor = df_participacao.filter(col("CO_PAIS") == 676).select(
    "NO_CUCI_DIVISAO", "perc_participacao_valor"
).orderBy(col("perc_participacao_valor").desc())

df_Russia_qtd = df_participacao.filter(col("CO_PAIS") == 676).select(
    "NO_CUCI_DIVISAO", "perc_participacao_qtd"
).orderBy(col("perc_participacao_qtd").desc())

# ========================================
# CONVERTENDO PARA PANDAS E CRIANDO GRÁFICOS
# ========================================

# 8. Converter para Pandas (Top 20)
df_valor_pd = df_Russia_valor.toPandas().head(20)
df_qtd_pd = df_Russia_qtd.toPandas().head(20)

# 9. Gráfico por Valor com cor escura (verde escuro)
fig_valor = go.Figure(data=go.Bar(
    x=df_valor_pd['perc_participacao_valor'],
    y=df_valor_pd['NO_CUCI_DIVISAO'],
    orientation='h',
    marker=dict(color='darkgreen'),
    text=df_valor_pd['perc_participacao_valor'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_valor.update_layout(
    title='Top 20 Produtos - Participação dos Russia nas Exportações por Valor (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_valor.show()

# 10. Gráfico por Quantidade com cor azul (steelblue)
fig_qtd = go.Figure(data=go.Bar(
    x=df_qtd_pd['perc_participacao_qtd'],
    y=df_qtd_pd['NO_CUCI_DIVISAO'],
    orientation='h',
    marker=dict(color='steelblue'),
    text=df_qtd_pd['perc_participacao_qtd'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_qtd.update_layout(
    title='Top 20 Produtos - Participação dos Russia nas Exportações por Quantidade (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_qtd.show()

# LIMPEZA DE MEMÓRIA
del df_join, df_pais, df_total, df_participacao, df_Russia_valor, df_Russia_qtd, df_valor_pd, df_qtd_pd, fig_valor, fig_qtd

In [ ]:
# 1. Carregar o dicionário de agregação
df_ncm = spark.read.parquet(f"{base_path}df_produtoscuci_master_select.parquet")

# 3. Join com dados de Importacao + filtro de período
df_join = df_import.join(
    df_ncm,
    df_import.SH4 == df_ncm.CO_SH4,
    "left"
).filter(
    ((col("CO_ANO") == 2024) & (col("CO_MES") >= 7)) |
    ((col("CO_ANO") == 2025) & (col("CO_MES") <= 6))
).select(
    "NO_CUCI_DIVISAO",
    "CO_PAIS",
    "VL_FOB",
    "KG_LIQUIDO"
)

# 4. Agregar total importado por produto e por país
df_pais = df_join.groupBy("NO_CUCI_DIVISAO", "CO_PAIS").agg(
    sum("VL_FOB").alias("vl_fob_pais"),
    sum("KG_LIQUIDO").alias("qtd_kgl_pais")
)

# 5. Agregar total importado por produto (todos os países)
df_total = df_join.groupBy("NO_CUCI_DIVISAO").agg(
    sum("VL_FOB").alias("vl_fob_total"),
    sum("KG_LIQUIDO").alias("qtd_kgl_total")
)

# 6. Juntar os dois e calcular a participação percentual
df_participacao = df_pais.join(
    df_total, on="NO_CUCI_DIVISAO"
).withColumn(
    "perc_participacao_valor", round(100 * col("vl_fob_pais") / col("vl_fob_total"), 1)
).withColumn(
    "perc_participacao_qtd", round(100 * col("qtd_kgl_pais") / col("qtd_kgl_total"), 1)
)

# 7. Filtrar só os Russia (código país = 676 no COMEX)

# Para Russia:
df_Russia_valor = df_participacao.filter(col("CO_PAIS") == 676).select(
    "NO_CUCI_DIVISAO", "perc_participacao_valor"
).orderBy(col("perc_participacao_valor").desc())

df_Russia_qtd = df_participacao.filter(col("CO_PAIS") == 676).select(
    "NO_CUCI_DIVISAO", "perc_participacao_qtd"
).orderBy(col("perc_participacao_qtd").desc())

# ========================================
# CONVERTENDO PARA PANDAS E CRIANDO GRÁFICOS
# ========================================

# 8. Converter para Pandas (Top 20)
df_valor_pd = df_Russia_valor.toPandas().head(20)
df_qtd_pd = df_Russia_qtd.toPandas().head(20)

# 9. Gráfico por Valor com cor escura (verde escuro)
fig_valor = go.Figure(data=go.Bar(
    x=df_valor_pd['perc_participacao_valor'],
    y=df_valor_pd['NO_CUCI_DIVISAO'],
    orientation='h',
    marker=dict(color='darkgreen'),
    text=df_valor_pd['perc_participacao_valor'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_valor.update_layout(
    title='Top 20 Produtos - Participação dos Russia nas Importações por Valor (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_valor.show()

# 10. Gráfico por Quantidade com cor azul (steelblue)
fig_qtd = go.Figure(data=go.Bar(
    x=df_qtd_pd['perc_participacao_qtd'],
    y=df_qtd_pd['NO_CUCI_DIVISAO'],
    orientation='h',
    marker=dict(color='steelblue'),
    text=df_qtd_pd['perc_participacao_qtd'].round(1).astype(str) + '%',
    textposition='outside'
))

fig_qtd.update_layout(
    title='Top 20 Produtos - Participação dos Russia nas Importações por Quantidade (%) (07/2024 - 06/2025)',
    xaxis_title='Participação (%)',
    yaxis_title='Produto',
    height=800,
    width=1400,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=15),
    title_x=0.5,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig_qtd.show()

# LIMPEZA DE MEMÓRIA
#del df_join, df_pais, df_total, df_participacao, df_Russia_valor, df_Russia_qtd, df_valor_pd, df_qtd_pd, fig_valor, fig_qtd


## Top 5 Produtos Importados da Rússia e Principais Fornecedores Alternativos

In [ ]:
# 1. Top 5 produtos importados da Rússia
top5_produtos = (
    df_participacao.filter(col("CO_PAIS") == 676)
    .groupBy("NO_CUCI_DIVISAO")
    .agg(sum("vl_fob_pais").alias("valor_total"))
    .orderBy(col("valor_total").desc())
    .limit(5)
    .select("NO_CUCI_DIVISAO")
    .rdd.flatMap(lambda x: x)
    .collect()
)

# 2. Países alternativos (≠ Rússia) que mais exportam cada produto (top 3)
df_alternativos = (
    df_participacao
    .filter((col("NO_CUCI_DIVISAO").isin(top5_produtos)) & (col("CO_PAIS") != 676))
)

# 3. Juntar nomes dos países
df_paises = spark.read.parquet(f"{base_path}df_paises_master.parquet")  # com CO_PAIS, NO_PAIS
df_alternativos = df_alternativos.join(df_paises, on="CO_PAIS", how="left")

# 4. Top 3 países por produto
window = Window.partitionBy("NO_CUCI_DIVISAO").orderBy(col("vl_fob_pais").desc())
df_top3_alternativos = (
    df_alternativos
    .withColumn("rank", row_number().over(window))
    .filter(col("rank") <= 3)
    .select("NO_CUCI_DIVISAO", "NO_PAIS", "vl_fob_pais")
)

# 5. Converter para pandas para Sankey
df_sankey = df_top3_alternativos.toPandas()

# 6. Criar lista única de nós (produtos à esquerda, países à direita)
produtos = df_sankey["NO_CUCI_DIVISAO"].unique()
paises = df_sankey["NO_PAIS"].unique()
nodes = list(produtos) + list(paises)
node_indices = {name: idx for idx, name in enumerate(nodes)}

# 7. Ajustar source/target: produtos (source), países (target)
df_sankey["source"] = df_sankey["NO_CUCI_DIVISAO"].map(node_indices)
df_sankey["target"] = df_sankey["NO_PAIS"].map(node_indices)

# 8. Gerar cores aleatórias para os links
import plotly.colors as Pastel
palette = Pastel.qualitative.Plotly  # ou 'D3', 'Set3', 'Pastel', etc.
link_colors = [palette[i % len(palette)] for i in range(len(df_sankey))]

# 9. Construir gráfico Sankey
fig = go.Figure(data=[go.Sankey(
    arrangement="snap",
    node=dict(
        pad=20,
        thickness=20,
        line=dict(color="black", width=0.6),
        label=nodes,
        color="lightsteelblue"
    ),
    link=dict(
        source=df_sankey["source"],
        target=df_sankey["target"],
        value=df_sankey["vl_fob_pais"],
        color=link_colors
    )
)])

# 10. Layout e título
fig.update_layout(
    title_text="Top 5 Produtos Importados da Rússia e Seus 3 Principais Fornecedores Alternativos",
    font_size=14,
    font_family="Arial",
    margin=dict(l=40, r=40, t=60, b=30),
    height=800,
    width=1200
)

fig.show()


____________________________________________
## Análise do Suco de Laranja

### Quais são os paises maiores compradores de Suco de Laranja do Brasil?

* O Brasil é atualmente o maior produtor e exportador de suco de laranja do planeta, vendendo 95% de sua produção para o exterior. Desse volume, xx% têm os Estados Unidos como destino.

* Atualmente, os americanos pagam uma tarifa fixa de US\$ 415 (equivalente a R\$ 2.296,32) por tonelada do suco brasileiro e mais uma taxa adicional de 10% (cerca de US\$ 308, ou R\$ 1.704,26 por tonelada), determinada por Trump em abril.

* Números dos EUA: O suco de laranja brasileiro, ele é responsável por 56% do consumo americano e 70% das importações.




fontes informações complementares:   

1. https://g1.globo.com/economia/agronegocios/noticia/2025/07/11/suco-de-laranja-e-substituivel-tarifa-de-trump-gera-incerteza-no-setor-mesmo-com-consumo-dos-eua-dependente-do-brasil.ghtml

2. https://cbn.globo.com/programas/estudio-cbn/entrevista/2025/07/30/setor-produtor-de-suco-de-laranja-comemora-excecao-em-tarifaco-de-trump.ghtml

In [ ]:
df_ncm_adicional = spark.read.parquet(f"{base_path}ncm_adicional.parquet")
df_paises_master = spark.read.parquet(f"{base_path}df_paises_master.parquet")

# 1. Criar coluna id_sh4 no dicionário
df_ncm_adicional = df_ncm_adicional.withColumn(
    "id_sh4",
    substring(regexp_replace(col("id_sh6").cast("string"), "\\.0$", ""), 1, 4).cast("int")
)

# 2. Join com os dados de exportação e filtro por suco de laranja (200911 e 200912) e período
df_suco = df_export.join(
    df_ncm_adicional,
    df_export.SH4 == df_ncm_adicional.id_sh4,
    "left"
).filter(
    (col("id_sh6").isin([200911, 200912])) &
    (((col("CO_ANO") == 2024) & (col("CO_MES") >= 7)) |
     ((col("CO_ANO") == 2025) & (col("CO_MES") <= 6)))
).select(
    "CO_PAIS",
    "VL_FOB",
    "KG_LIQUIDO"
)

# 3. Agregar total exportado por país
df_top_paises = df_suco.groupBy("CO_PAIS").agg(
    round(sum("VL_FOB"), 2).alias("vl_fob_total"),
    round(sum("KG_LIQUIDO"), 2).alias("kg_total")
)

# 4. Reduz o df_paises_master para conter apenas um nome por país
df_paises_unicos = df_paises_master.groupBy("CO_PAIS").agg(
    first("NO_PAIS").alias("NO_PAIS")
)

# 5. Join com nomes únicos
df_top_paises_nome = df_top_paises.join(
    df_paises_unicos,
    on="CO_PAIS",
    how="left"
).select(
    "NO_PAIS", "vl_fob_total", "kg_total"
).orderBy(col("kg_total").desc())

# 6. Mostrar resultado final
df_top_paises_nome.show(10, truncate=False)

# Convertendo DataFrame Spark para Pandas
df_suco_pd = df_top_paises_nome.toPandas()

# Pegar top 10 países para melhor visualização
df_top10 = df_suco_pd.head(10)

# 1. Gráfico por Quantidade (KG) - Principais Importadores
fig_qtd = go.Figure(data=go.Bar(
    x=df_top10['kg_total'],
    y=df_top10['NO_PAIS'],
    orientation='h',
    marker=dict(
        color=df_top10['kg_total'],
        colorscale='Turbo',
        cmin=0,
        cmax=df_top10['kg_total'].max(),
        colorbar=dict(title="Quantidade (KG)")
    ),
    text=[f"{val:,.0f}" for val in df_top10['kg_total']],
    textposition='outside'
))

fig_qtd.update_layout(
    title='Top 10 Países - Importadores de Suco de Laranja Brasileiro (Quantidade)',
    xaxis_title='Quantidade (KG)',
    yaxis_title='País',
    height=600,
    width= 1200,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=12),
    title_x=0.5
)

fig_qtd.show()

# 2. Gráfico por Valor (FOB) - Principais Importadores
# Ordenar por valor para este gráfico
df_top10_valor = df_suco_pd.sort_values('vl_fob_total', ascending=True).tail(10)

fig_valor = go.Figure(data=go.Bar(
    x=df_top10_valor['vl_fob_total'],
    y=df_top10_valor['NO_PAIS'],
    orientation='h',
    marker=dict(
        color=df_top10_valor['vl_fob_total'],
        colorscale='Turbo',
        cmin=0,
        cmax=df_top10_valor['vl_fob_total'].max(),
        colorbar=dict(title="Valor FOB (USD)")
    ),
    text=[f"${val:,.0f}" for val in df_top10_valor['vl_fob_total']],
    textposition='outside'
))

fig_valor.update_layout(
    title='Top 10 Países - Importadores de Suco de Laranja Brasileiro (Valor FOB)',
    xaxis_title='Valor FOB (USD)',
    yaxis_title='País',
    height=600,
    width= 1200,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=12),
    title_x=0.5
)

fig_valor.show()

# Opcional: Salvar os gráficos
# fig_qtd.write_html("suco_quantidade.html")
# fig_valor.write_html("suco_valor.html")

+-----------------------+------------+----------+
|NO_PAIS                |vl_fob_total|kg_total  |
+-----------------------+------------+----------+
|Estados Unidos         |2989347256  |2551636456|
|Bélgica                |2184654460  |971820078 |
|Países Baixos (Holanda)|1471850748  |876327478 |
|China                  |215970948   |66394590  |
|Japão                  |271727496   |53151956  |
|Espanha                |42126102    |30634866  |
|Chile                  |60838654    |14551720  |
|Austrália              |36528228    |11460676  |
|Israel                 |40341826    |9590208   |
|Canadá                 |8258248     |8486734   |
+-----------------------+------------+----------+
only showing top 10 rows

